In [2]:
# Sample message from testing set
message = 'You have WON a guaranteed £1000 cash or a £2000 prize.To claim yr prize call our customer service representative on'

# Feature Engineering

In [3]:
import string

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count / (len(text) - text.count(" ")), 3) * 100

In [4]:
message_length = len(message) - message.count(" ")
punctuation_percentage = count_punct(message)
message_length, punctuation_percentage

(96, 1.0)

# Clean data

In [5]:
import nltk
import re

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# Vectorize Data

In [6]:
import joblib

vect = joblib.load('tfidf_vect_fit.pkl')

In [7]:
import pandas as pd

tfidf_message = vect.transform(pd.DataFrame(data={'message':[message]})['message'])

message_vect = pd.concat([pd.DataFrame([{'message_length': message_length, 'punct%': punctuation_percentage}]),
                         pd.DataFrame(tfidf_message.toarray(), columns=vect.get_feature_names_out())], axis=1)

message_vect.head()

,message_length,punct%,,0,008704050406,0121,01223585236,01223585334,0125698789,020603,...,zhong,zindgi,zoe,zogtoriu,zoom,zouk,zyada,ü,üll,〨ud
0,96,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Predict

In [8]:
rf_model = joblib.load('rf_model.pkl')
y_pred = rf_model.predict(message_vect)

y_pred[0]

'spam'

# Create function to classify messages

In [17]:
message = 'No just send to you. Bec you in temple na.'

In [20]:
def classify(text_message, vectorizer, model):

    # Engineer Features
    text_length = len(message) - message.count(" ")
    percent_punctuation = count_punct(text_message)

    # Vectorize Text
    tfidf_vect_message = vectorizer.transform(pd.DataFrame(data={'message':[text_message]})['message'])

    # Combine engineered features with vectorized text
    message_features = pd.concat([pd.DataFrame([{'message_length': text_length, 'punct%': percent_punctuation}]),
                                  pd.DataFrame(tfidf_vect_message.toarray(), columns=vectorizer.get_feature_names_out())], axis=1)

    # Predict
    prediction = model.predict(message_features)[0]

    return prediction

print(message, 'is:', classify(message, vect, rf_model))

No just send to you. Bec you in temple na. is: ham
